<a href="https://colab.research.google.com/github/TirendazAcademy/Audio-Data-with-HuggingFace/blob/main/3-Music-Genre-Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing

Let's install the trasformers library.

In [ ]:
!pip install -qU git+https://github.com/huggingface/transformers
!pip install -qU git+https://github.com/huggingface/datasets

In [ ]:
import transformers, datasets
print("The transformers version:", transformers.__version__)
print("The datasets version:", datasets.__version__)

Next, let's load the dataset we'll use.

In [ ]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

#  Pre-trained models and datasets for audio classification

Then let's create our pipeline:

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

Finally, we can pass a sample to the classification pipeline to make a prediction:

In [ ]:
classifier(minds[0]["audio"])

## Speech Commands

First, let's load the dataset.

In [ ]:
speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)

In [ ]:
sample = next(iter(speech_commands))

In [ ]:
sample

 Let's load an official Audio Spectrogram Transformer checkpoint fine-tuned on the Speech Commands dataset

In [ ]:
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"].copy())

As you can see, the prediction of our model is backward. Let me take a listen to the sample and verify this is prediction:

In [ ]:
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

## Language Identification

Language identification (LID) is the task of identifying the language spoken in an audio sample from a list of candidate languages systems in 102 languages.

In [ ]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True, trust_remote_code=True)
sample = next(iter(fleurs))

Next, let's create our pipeline.

In [ ]:
classifier = pipeline(
    "audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id"
)

We can then pass the audio through our classifier and generate a prediction:

In [ ]:
classifier(sample["audio"])

## Zero-Shot Audio Classification

In [ ]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
audio_sample = next(iter(dataset))["audio"]["array"]

In [ ]:
candidate_labels = ["Sound of a dog", "Sound of vacuum cleaner"]

In [ ]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=candidate_labels)

In [ ]:
Audio(audio_sample, rate=16000)

# Fine-tuning a model for music classification

In [ ]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
gtzan

There is no a test set. Let's create one.

In [ ]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

In [ ]:
gtzan["train"][0]

Let’s use the int2str() method of the genre feature to map these integers to human-readable names:

In [ ]:
gtzan["train"].features["genre"]

In [ ]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

Let’s now listen to a few more examples by using Gradio

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

## From audio to machine learning features

###  Preprocessing the data

Let’s begin by instantiating the feature extractor for DistilHuBERT from the pre-trained checkpoint:

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

We’ll have to resample the audio file to 16,000 Hz before passing it to the feature extractor.

In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

In [ ]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

We can now check the first sample of the train-split of our dataset to verify that it is indeed at 16,000 Hz.

In [ ]:
gtzan["train"][0]

Let's take a look at the feature extractor in operation by applying it to our first audio sample. First, let’s compute the mean and variance of our raw audio data:

In [ ]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Let’s apply the feature extractor and see what the outputs look like:

In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

Let's truncate the audio clips to be 30 seconds in length.

In [ ]:
max_duration = 30.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

 We can now apply it to the dataset using the map() method.

In [ ]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Finally, we need to obtain the label mappings from the dataset.

In [ ]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

OK, we’ve now got a dataset that’s ready for training! Let’s take a look at how we can train a model on this dataset.

## Fine-tuning the model

To fine-tune the model, we’ll use the Trainer class from 🤗 Transformers

In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_id

In [ ]:
model_id.split("/")[-1]

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

Let's define the metrics.

In [ ]:
!pip install -q evaluate

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

Let’s instantiate the Trainer and train the model:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

## Uploading the hub

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan-evren",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)

## Sharing the model

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "audio-classification", model="sanchit-gandhi/distilhubert-finetuned-gtzan"
)